In [ ]:

import pandas as pd
import numpy as np
from bertopic import BERTopic
from datetime import datetime
from umap import UMAP
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

In [ ]:
# =============================================================================
# SEZIONE 1: Preparazione dati e parametri
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# 1) Imposta i parametri di input per l'analisi (tipo di dati testuali da utilizzare (nell'esempio Titolo e descrizione) e numero di categorie (nell'esempio 100 categorie)).
# 2) Carica il dataset delle azioni da CSV.
# 3) Crea una cartella di output con timestamp per salvare i risultati in modo univoco.
# 4) Filtra le righe con il campo 'descrizione' non vuoto.
# 5) Costruisce una nuova colonna testuale 'text' combinando 'titolo' e 'descrizione'.
# =============================================================================


# 1)--- Inputs ---
TYPE_OF_ANALYSIS = "TopicModelling_TitoloDescrizione_"  # in questa analisi usiamo sia titolo che descrizione. 
                                          # È possibile aggiungere altri campi, ad es. "obiettivi".
N_TOPICS = 100  # numero di topic/categorie che il modello deve trovare; regolarlo in base alla granularità desiderata.

# 2) --- Caricamento dati ---
df = pd.read_csv("./FT/azioni.csv") # aggiustare il percorso del file CSV se necessario

# 3) 
# --- Timestamp per cartella risultati (es. 20251030-1423) ---
stamp = datetime.now().strftime("%Y%m%d-%H%M")

# --- Nome cartella ---
folder_name = f"{stamp}{TYPE_OF_ANALYSIS}n{N_TOPICS}"

# --- Creazione directory dei risultati ---

mydir= './'+TYPE_OF_ANALYSIS+stamp + '_'+ str(N_TOPICS)+'topics/' # nel github per comodità la cartella è chiamata semplicemente "Results"


os.mkdir(mydir)
print("Created:", mydir)

# 4) --- Filtro e costruzione ---
# Mantiene solo le righe con 'descrizione' non nulla
df_filtered = df.dropna(subset=['descrizione']).copy()

# 5) --- Filtro e costruzione ---
# Crea la colonna 'text' combinando 'titolo' e 'descrizione' (robusto a NaN/whitespace)
df_filtered['text'] = (
    'Titolo: '
    + df_filtered['titolo'].fillna('').astype(str).str.strip()
    + '; Descrizione: '
    + df_filtered['descrizione'].astype(str).str.strip()
)

print("Totale documenti (titolo azione + descrizione) utilizzati per l'analisi:", len(df_filtered))

In [ ]:
# =============================================================================
# SEZIONE 2: Pulizia del testo (italiano) – mantiene la punteggiatura
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# - Tokenizza il testo con NLTK `word_tokenize` (mantiene la punteggiatura).
# - Rimuove le stopword italiane di NLTK (confronto case-insensitive).
# - Restituisce una lista di stringhe pulite da usare, ad esempio, con BERTopic.
# Nota: non forza il lowercase dell’output e non rimuove la punteggiatura.
# =============================================================================

nltk.download('stopwords', quiet=True)

STOP_WORDS = set(stopwords.words('italian'))

def drop_stopwords(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    return " ".join(w for w in tokens if w.lower() not in STOP_WORDS)

# Applica alla colonna e ottiene una lista
docs_unique = df_filtered['text'].fillna('').map(drop_stopwords).tolist()

print(docs_unique[10])  # esempio di documento pulito


In [ ]:
# =============================================================================
# SEZIONE 3: creazione del modello con BERTopic e riduzione dimensionale al numero di categorie desiderate
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# - Configura UMAP per ridurre la dimensionalità delle embedding testuali.
# - Inizializza BERTopic per lingua italiana con i parametri dell’analisi.
# - Addestra il modello sui documenti e restituisce l’assegnazione dei topic.
# - Estrae gli embedding dei documenti dal modello addestrato e li riduce a 2 dimensioni per la visualizzazione (es. scatter).
# =============================================================================

# UMAP per la riduzione dimensionale interna usata da BERTopic
umap_model = UMAP(
    n_neighbors=15,      # equilibrio tra struttura locale/globale
    n_components=5,      # dimensione target per la clusterizzazione
    min_dist=0.0,        # addensa i punti; utile per separare i cluster
    metric='cosine',     # adatta a embedding testuali
    random_state=42      # riproducibilità
)

# Inizializza BERTopic con parametri principali dell'analisi
topic_model = BERTopic(
    language="multilingual",          # italian is not supported    
    calculate_probabilities=False,   # più veloce, meno memoria
    verbose=True,                    
    nr_topics=N_TOPICS,              # numero di topic desiderato
    top_n_words=30,                  # n, parole rappresentative mostrate per topic
    umap_model=umap_model            # UMAP definito sopra per ri
)

# Addestra il modello e ottiene:
# - topics: indice di topic per ciascun documento
topics, prob = topic_model.fit_transform(docs_unique)

# Estrae gli embedding dei documenti e riduzione a 2D per plotting/visualizzazione
embeddings = topic_model._extract_embeddings(docs_unique, method="document")
reduced_embeddings = UMAP(
    n_neighbors=15,
    n_components=2,
    min_dist=0.0,
    metric='cosine',
    random_state=42
).fit_transform(embeddings)


In [ ]:
# =============================================================================
# SEZIONE 4: Salvataggio del modello e dei risultati (BERTopic)
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# - Crea la cartella del modello e salva il modello BERTopic (CTFIDF + embedding model).
# - Esporta l’assegnazione documento→topic.
# - Esporta le informazioni riassuntive dei topic (descrizioni, conteggi, ecc.).
# - Unisce i metadati originali dei documenti con le info dei topic e salva i risultati.
# =============================================================================


# --- Cartella modello ---

model_dir = mydir +"BERTopicModel"
os.makedirs(model_dir, exist_ok=True)
# Modello di embedding usato, in questo caso è il default (deve corrispondere a quello del training)
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

# --- Salvataggio modello BERTopic ---
# serialization="pytorch": salva i pesi in formato torch
# save_ctfidf=True: salva la matrice C-TF-IDF (utile per ricaricare topic/termini)
# save_embedding_model=...: salva anche il modello di embedding
topic_model.save(
    str(model_dir),
    serialization="pytorch",
    save_ctfidf=True,
    save_embedding_model=embedding_model
)

# --- Esporta informazioni sui topic ---
# get_topic_info() restituisce, tra le altre, le colonne: Topic, Count, Name

topic_info = topic_model.get_topic_info()
del topic_info['Representative_Docs']  # rimuove colonna non necessari per avere un ouput più leggero
topic_info= topic_info.drop(df.index[0]) #la prima riga si riferisce a topic -1 (outliers non assegnati a nessuna categoria), non di interesse per successive analisi 
topic_info.to_csv(mydir +"topics_overview.tsv", sep="\t", index=False)


In [ ]:
# =============================================================================
# SEZIONE 5: Gerarchia dei topic (dendrogramma).
# Ispezione visiva delle relazioni tra i topic e i nomi dei topic. Utile anche per effettuare un sanity check dell'analisi
# -----------------------------------------------------------------------------
# Questa sezione:
# - Genera il dendrogramma gerarchico dei topic individuati da BERTopic.
# - Salva la stessa visualizzazione in un file HTML interattivo.
# - Salva la struttura del dendrogramma in un file di testo.
# =============================================================================

# Mostra il dendrogramma nel notebook 
hierarchical_topics = topic_model.hierarchical_topics(docs_unique)
hierarchical_topics.to_csv(mydir + "hierarchical_topics.csv")

tree = topic_model.get_topic_tree(hierarchical_topics)
f_out = open(mydir+'/topics_tree.txt','w')
f_out.write(tree)

# Crea la figura del dendrogramma (identica a quella mostrata sopra)
fig1 = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

# Salva la figura come HTML interattivo nella cartella dei risultati.
fig1.write_html(mydir + "/Dendrogram_taxonomy.html")
